Cập nhật 23/10/2020

# Tiếp tục ví dụ buổi trước

In [1]:
import requests
from bs4 import BeautifulSoup

In [2]:
# Lấy nội dung html của trang chứa các kết quả tuyển dụng 
# khi tìm kiếm trên trang careerbuilder bằng từ khóa data scientist
url = 'https://careerbuilder.vn/viec-lam/data-scientist-k-vi.html'
html_text = requests.get(url).text

In [3]:
# Parse html
tree = BeautifulSoup(html_text, 'html.parser')

In [4]:
# Tìm kiếm các tag ứng với các tuyển dụng
jobs = tree.find_all('div', {'class': 'job-item'})
len(jobs)

11

In [6]:
# Với mỗi tag tuyển dụng, rút trích: 
# tựa đề, công ty tuyển dụng, lương, địa điểm, ngày đăng thông báo, 
# link đến nội dung chi tiết, nội dung chi tiết
# Đồng thời, ghi các thông tin này xuống file csv/tsv, mỗi tuyển dụng là một dòng
# Ở đây, ta sẽ dùng file tsv (tab được dùng để phân tách các giá trị trên một dòng)
# vì dấu phẩy nhiều khả năng là sẽ xuất hiện trong các giá trị

file = open('03_result.csv', 'w', encoding='utf-8') # Bạn cần tạo thư mục Data cùng cấp với file notebook này
count = 1
for job in jobs:
    print(count); count += 1
    title = job.find('a', {'class': 'job_link'})['title'] # Nếu dùng .string.strip() thì có một tuyển dụng bị lỗi tại thời điểm mình chạy    
    company = job.find('a', {'class': 'company-name'}).string
    salary = job.find('div', {'class': 'salary'}).text
    location = job.find('div', {'class': 'location'}).text.strip()
    posted_date = job.find('time').string
    url = job.find('a', {'class': 'job_link'})['href']
    
    # Vào link tuyển dụng để lấy nội dung chi tiết
    job_html_text = requests.get(url).text
    job_tree = BeautifulSoup(job_html_text, 'html.parser')
    ## Ta cần thay ký tự xuống dòng trong chuỗi detail bằng một ký tự nào đó
    ## như '>>' vì nếu không thì ký tự xuống dòng sẽ làm loạn file csv
    details = job_tree.get_text(separator='>>', strip=True)
    ## Fix lỗi mà mình nói ở dưới
    details = details.replace('\n\r', '>>')
    details = details.replace('\n', '>>')
    details = details.replace('\r', '>>')
    
    file.write(f'{title}\t{company}\t{salary}\t{location}\t{posted_date}\t{url}\t{details}\n')
    
file.close()
print('Done!')

1
2
3
4
5
6
7
8
9
10
11
Done!


## Đếm số lần xuất xuất hiện của các ngôn ngữ lập trình

In [6]:
# Đầu tiên, đọc dữ liệu đã lưu xuống file 
file = open('Data/ds_jobs.tsv', 'r', encoding='utf-8')
job_lines = file.readlines()
file.close()

In [7]:
# Số lượng dòng đọc được phải bằng số lượng tuyển dụng
len(job_lines)

13

Rất tiếc là không bằng! Đây chính là lỗi mà mình đang chưa fix được khi demo trên lớp. Đối với người lập trình thì mình nghĩ lỗi là một điều tốt 🙂, vì thông quá trình fix lỗi thì kinh nghiệm sẽ được tăng lên.

Sau khi xem xét thì mình phát hiện ra lỗi là do trong chuỗi chứa nội dung chi tiết tuyển dụng có ký tự `'\r'` (mình nghĩ có thể là `'\r'` trong `'\r\n'`, cũng có thể là `'\r'` đứng một mình). Mặc định thì phương thức `readlines` của Python sẽ xem `'\r'` hay `'\n'` hay `'\r\n'` đều là ký tự xuống dòng và sẽ chuyển thành `'\n'`.

Bài tập trong tuần này của bạn là:
1. Sửa lỗi ở trên
2. Sau đó, với mỗi phần tử (mỗi dòng) trong list `job_lines` ở trên, bạn lấy ra chuỗi nội dung chi tiết của tuyển dụng, chuẩn hóa bằng cách viết thường hết, và dùng Regex để rút trích tập từ; rồi bạn check các ngôn ngữ lập trình có trong tập từ này không (ở đây, bạn xét các ngôn ngữ sau: python, r, matlab, c, c++, c#, java), nếu có thì cộng biến đếm của ngôn ngữ đó lên 1.

In [8]:
import re

In [9]:
language_counts = {'python': 0, 'r': 0, 'c': 0, 'c++': 0, 'c#': 0, 'java': 0}
for job_line in job_lines:
    details = job_line[:-1].split('\t')[-1]
    details = details.lower()
    word_set = set(re.findall(r'[\w+#]+', details))
    for language in language_counts:
        if language in word_set:
            language_counts[language] += 1

language_counts

{'python': 6, 'r': 3, 'c': 0, 'c++': 0, 'c#': 0, 'java': 2}

# Trang web có JavaScript

In [10]:
html_text = requests.get('http://avi.im/stuff/js-or-no-js.html').text

In [11]:
tree = BeautifulSoup(html_text, 'html.parser')

In [12]:
tree.find_all(attrs={'id': 'intro-text'})

[<p id="intro-text">No javascript support</p>]

In [13]:
from selenium import webdriver

In [14]:
browser = webdriver.Chrome(executable_path='./chromedriver.exe') 
# Bạn cần để file "chromedriver.exe" cùng cấp với thư mục chứa file notebook này

In [15]:
browser.get('http://avi.im/stuff/js-or-no-js.html')

In [16]:
html_text = browser.page_source
print(html_text)

<html><head>
  <meta charset="utf-8">
  <title>Javascript scraping test</title>
</head>
<body>
  <p id="intro-text">Yay! Supports javascript</p>
<script type="text/javascript">
    document.getElementById('intro-text').innerHTML = 'Yay! Supports javascript';
</script>

</body></html>


In [17]:
browser.quit()

In [18]:
tree = BeautifulSoup(html_text)
tree.find_all(attrs={'id': 'intro-text'})

[<p id="intro-text">Yay! Supports javascript</p>]

# Trang web đòi hỏi phải tương tác

In [19]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

In [20]:
# Ngày mai là ngày bao nhiêu?
from datetime import date, timedelta

today = date.today()
tomorrow = today + timedelta(1)
print(tomorrow)
tomorrow_day = str(tomorrow.day)
tomorow_month = tomorrow.strftime('%b')
print(tomorrow_day, tomorow_month)

2020-10-25
25 Oct


In [21]:
# Mở Chrome và cho full màn hình
browser = webdriver.Chrome(executable_path='./chromedriver.exe')
browser.maximize_window()

In [22]:
# Mở trang chủ của Vietjet
browser.get('https://www.vietjetair.com/Sites/Web/vi-VN/Home')

In [23]:
# Tắt cửa sổ quảng cáo nếu có bằng cách ấn phím Esc 
browser.find_element_by_css_selector('html').send_keys(Keys.ESCAPE)

In [24]:
# Chọn điểm khởi hành là TP HCM 
locations = browser.find_elements_by_css_selector('#selectOrigin option')
for location in locations:
    if 'SGN' in location.text:
        location.click()
        break

In [25]:
# Chọn điểm đến là Đà Nẵng
locations = browser.find_elements_by_css_selector('#selectDestination option')
for location in locations:
    if "DAD" in location.text:
        location.click()
        break

In [26]:
# Chọn mua vé một chiều
browser.find_element_by_css_selector('#ctl00_UcRightV31_RbOneWay').click()

In [27]:
# Click chỗ ngày đi để hiện lên calendar để chọn
browser.find_element_by_css_selector('#ctl00_UcRightV31_TxtDepartDate').click()

In [28]:
# Chọn tháng đi
months = browser.find_elements_by_css_selector('#ui-datepicker-div option')
for month in months:
    if month.text == tomorow_month:
        month.click()
        break

In [29]:
# Chọn ngày đi
days = browser.find_elements_by_css_selector('#ui-datepicker-div td')
for day in days:
    if day.text == tomorrow_day:
        day.click()
        break

In [30]:
# Ấn nút tìm chuyến bay
browser.find_element_by_css_selector('#ctl00_UcRightV31_BtSearch').click()

In [31]:
# Giao lại cho Beautiful Soup
html_text = browser.page_source
browser.quit()
tree = BeautifulSoup(html_text, 'html.parser')
# ...